# Customize

## Change website theme

If you want to change site colors and other changes, copy/edit [_static/css/jupman-web.css](https://github.com/DavidLeoni/jupman/blob/master/_static/css/jupman-web.css)  and set it in conf `html_css_files`:

```python
html_css_files = [
    'css/jupman.css',      # shared among jupyter and website
    'css/jupman-web.css',  # only on website
    #'css/softpython-theme.css',  # uncomment to activate
    #'css/scifi-theme.css',
]

```

## Fonts

Fonts are a bit of a complex topic

TODO this part is just a collection of personal notes


* [The missing guide to font formats](https://creativemarket.com/blog/the-missing-guide-to-font-formats)

* https://docs.readthedocs.io/en/latest/guides/adding-custom-css.html

* [RTD Code font issue on github](https://github.com/readthedocs/sphinx_rtd_theme/issues/524)


Tools: 

Comprehensive article: https://www.useragentman.com/blog/2011/02/20/converting-font-face-fonts-quickly-in-any-os/
and https://www.useragentman.com/blog/the-css3-font-converter/


https://github.com/zoltan-dulac/css3FontConverter

woff2

https://github.com/google/woff2

sfnt2woff

```bash
sudo apt-get install  libbrotli-dev
sfnt2woff  SomeFont.otf
```

mkeot

```bash
sudo apt-get install eot-utils
mkeot SomeFont.otf > SomeFont.eot
```

or https://github.com/wget/ttf2eot

FontForge (GUI and scriptable)

`sudo apt-get install fontforge`


## font sizes

https://www.24a11y.com/2019/pixels-vs-relative-units-in-css-why-its-still-a-big-deal/

https://chiamakaikeanyi.dev/sizing-in-css-px-vs-em-vs-rem/


## Warning about old versions

[ReadTheDocs has a mechanism](https://docs.readthedocs.io/en/latest/versions.html) to warn the user if he's looking at an old version of the site, but we found it doesn't work much for course-based documentation. So for versioning we think it's better to adopt a mixed git branch / tags devlopment model, and we added a template warning to show in old branches. To enable it in an old branch, just rename [_templates/breadcrumbs.html.bak](../_templates/breadcrumbs.html.bak) into `_templates/breadcrumbs.html` and edit as needed.


## Fix nbsphinx to create rst files 

Sometimes nbsphinx does not report properly RST conversion errors ([see bug](https://github.com/DavidLeoni/jupman/issues/9)). As a hacky workaround, you might take the `nbsphinx.py` from `~/.local/lib/python3.5/site-packages/` , make a copy of it in your project home and patch it  [like this](https://github.com/DavidLeoni/jupman/commit/0f332629ce4e2b0186c954c55aea7fa67992ace9#diff-bd3d9c4d2e80ed83fd2443d1301aa65bR649) 
When you call sphinx, it will generate RST files in `_build/jupman-rst/`.

Of course, things can be cleaner using a virtual env [with venv](https://docs.python.org/3/library/venv.html)

## Git performance notes

Current suggested setup for hosting on Github is creating branch gh-pages and using Github Actions to populate it with html, zips, pdf and epub files. While keeping all that stuff versioned may seem pretty inefficient, apparently git is [pretty good](https://stackoverflow.com/a/48305739) at compressing binary files 

The size of `.git` repo for a 1000 pdf page project SoftPython with 300 commits and 100 MB of code is:

```
.git: 183 MB
```

By truncating gh-pages to last commit and garbage collecting, we get:

```
.git: 139 MB
```

If we completely remove gh-pages branch, we get:

```
.git: 68.7 MB
```

So gh-pages size is:

- one commit: 70.3 MB
- 300 commits: 114.3 MB

which is not even double than source code git size. 

If the repo gets really huge, in order to to shrink it some git knowledge is required.

**If the repo is served from another server** and you want to truncate that server git repo:

On that server console:

1. first make sure you are on `gh-pages` branch:

```bash
git checkout gh-pages
```

2. truncates previous commits:
```bash
git fetch --depth=1 origin gh-pages
```

3. removes various links around which may still point to old commits:

```bash
git reflog expire --expire-unreachable=now --all
```

4. actually deletes from disk old commits:

```bash
git gc --aggressive --prune=all 
```

Note the result of truncation cannot be pushed back to origin as git would complain it is a _shallow_ branch.